In [2]:
#import fasttext
#pip install fasttext-0.9.1-cp37-cp37m-win32.whl 原本安裝fasttext要安裝C++ 容量太大太麻煩 安裝指令那一格不能有註解..........
#pip install wordcloud-1.6.0-cp37-cp37m-win32.whl
#https://www.lfd.uci.edu/~gohlke/pythonlibs/#fasttext  這邊抓東西可免去環境需要的安裝
from gensim.models.fasttext import FastText
from gensim.models import word2vec
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os,json,wordcloud
#讀取檔案
clean_for_fasttext_path_ingredient_names=r"F:\資策會\專題\爬蟲\venv\clean_for_w2v.txt"
clean_for_fasttext_path_step=r"F:\資策會\專題\爬蟲\venv\w2v_allpeople_cut_1230cleanstep.txt"
clean_for_word_embedding_path=r"F:\資策會\專題\爬蟲\venv\word_cut_1230cleanstep.txt"
wordcloud_path=r"F:\資策會\專題\爬蟲\venv\w2v_cut_1230cleanstep.txt"
path=r"F:\資策會\專題\爬蟲\venv\collction_freefood_jay_clean"

#設定要train的model
model_name_to_train="test_fasttext.bin"
#設定要用的model
model_name_to_use="test_fasttext.bin"

In [18]:
#從原始檔讀上食譜
def read_from_json():
    json_list = os.listdir(path)
    list_for_json=[]
    #29101~35529
    for i in range(29101,35530):
        with open(path + "/" +"food_json_"+str(i)+".json", "r", encoding="utf-8") as f:
            dic = f.read()
            d = json.loads(dic)
            list_for_json.append(d)
    return list_for_json

In [19]:
#從原始檔抓出ID跟食譜名稱
def take_name_ID_form_json(list_for_json):
    for n,d in enumerate(list_for_json):
        recipe_name=d["recipe_name"]
        recipe_name=recipe_name.replace(" ","")
        food_id = d["_id"]
        ID_name_vector.loc[n]=[recipe_name,food_id]

In [22]:
def show_similar(model):
    #print(model.get_nearest_neighbors("你"))
    list_test=["雞蛋","豬肉","中筋麵粉","番茄","煎","水","蒸","高麗菜","削","巧克力"]
    #list_test=["雞蛋","豬肉","中筋麵粉","番茄","水","高麗菜"]
    df=pd.DataFrame(columns=list_test)
    for n,i in enumerate(list_test):
        result = model.most_similar(i, topn=7)    
        df[i]=result
    print(df)

In [ ]:
# def prepare_for_fast_data():
#     data_list=[]
#     with open(clean_for_word_embedding_path,"r", encoding='utf-8') as f:
#         cut_step_every_food=f.read()
#         list_cut_step_every_food=cut_step_every_food.split("\n")
#         #刪去多的換行符號        
#         del list_cut_step_every_food[-1]
#         #每篇文章    
#         for n,cut_step_one_food in enumerate(list_cut_step_every_food):
#             #每個步驟的字,把字放進list
#             list_cut_step_one_food=cut_step_one_food.split(",")
#             data_list.append(list_cut_step_one_food)
#     #print(data_list)
#     return data_list
#原本以為一定要list裡面list才能用

In [20]:
def train_model(path):
    sentences = word2vec.LineSentence(path)
    #data=prepare_for_fast_data()
    model = FastText(sentences=sentences,size=17, window=5,workers=20 ,min_count=3,sg=0,iter=10)
    model.save(model_name_to_train)
    return model

In [21]:
def load_model():
    return FastText.load(model_name_to_use)

In [23]:
def train_by_step_show(clean_for_fasttext_path_step):
    model=train_model(clean_for_fasttext_path_step)
    #model=load_model()
    show_similar(model)
train_by_step_show(clean_for_fasttext_path_step)

f:\python3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


                           雞蛋                           豬肉  \
0     (蛋, 0.9094400405883789)    (豬肉包, 0.9525145292282104)   
1   (雞蛋黃, 0.8819732666015625)  (豬肉火鍋片, 0.8821613788604736)   
2  (水煮蛋黃, 0.8614808320999146)   (梅花絞肉, 0.8819665312767029)   
3   (雞蛋白, 0.8264668583869934)   (五花絞肉, 0.8770784139633179)   
4    (全蛋, 0.8069368600845337)  (梅花豬肉片, 0.8641852140426636)   
5  (馬鈴薯粉, 0.7846250534057617)   (梅花肉片, 0.8547980785369873)   
6    (打進, 0.7765377759933472)   (豬肉肉排, 0.8495012521743774)   

                         中筋麵粉                         番茄  \
0   (筋麵粉, 0.9449715614318848)   (蕃茄, 0.9468469023704529)   
1  (低筋麵粉, 0.9013145565986633)  (牛番茄, 0.9156501889228821)   
2  (高筋麵粉, 0.8838521838188171)  (綠番茄, 0.9111796021461487)   
3  (全麥麵粉, 0.8811448216438293)  (黃番茄, 0.9029255509376526)   
4    (麵粉, 0.8752610683441162)   (西洋芹, 0.901302695274353)   
5    (中粉, 0.8647719025611877)  (番茄冰, 0.8951805830001831)   
6    (中筋, 0.8528096675872803)  (番茄絲, 0.8934014439582825)   

                      

In [24]:
def calculator_food_vector():
    model=load_model()
    with open(clean_for_word_embedding_path,"r", encoding='utf-8') as f:
        cut_step_every_food=f.read()
        list_cut_step_every_food=cut_step_every_food.split("\n")
        #刪去多的換行符號        
        del list_cut_step_every_food[-1]
        #每篇文章
        every_avg_vector_list=[]       
        for n,cut_step_one_food in enumerate(list_cut_step_every_food):
            #每個步驟的字,把字放進list
            list_cut_step_one_food=cut_step_one_food.split(",")
            food_vector=np.zeros((1,17),dtype=np.float32)
            error_count=0                       
            for m,word in enumerate(list_cut_step_one_food):
                try:
                    food_vector+=np.matrix(model[word])
                #要先執行model[word] 才能觸發ERROR 不然count 都會是0
                except KeyError as e:
                    error_count+=1
            #food_vector=np.power(food_vector,1)
            ave_food_vector=food_vector/(len(list_cut_step_one_food)-error_count)
            #ave_food_vector=np.power(ave_food_vector,2)
            #ave_food_vector=np.around(ave_food_vector,decimals=4)
            
            #發現要PCA有空值回頭來檢查向量輸出
#             if np.isnan(ave_food_vector)=True:               
#             print(n,np.where(np.isnan(ave_food_vector)))
            every_avg_vector_list.append(ave_food_vector)         
        #用numpy推疊方式，堆出全部的,記憶體不足 不能用這個方法
#         for n,i in enumerate(every_avg_vector_list):
#             if n<len(every_avg_vector_list)-1:
#                 every_avg_vector_list[n+1]=np.vstack((every_avg_vector_list[n],every_avg_vector_list[n+1]))
#                 final=every_avg_vector_list[n+1]
#         print(final)
        con=np.zeros((len(every_avg_vector_list),17),dtype="float32")        
        for n,i in enumerate(every_avg_vector_list):
            A=con[n]
            A[:]=i
        return con


In [25]:
ID_name_vector=pd.DataFrame(columns=["name","food_id"])
list_for_json=read_from_json()
take_name_ID_form_json(list_for_json)
every_food_numpy_vector=calculator_food_vector()
vector_name_list=["v"+str(i) for i in range(1,18) ]
every_food_df_vector=pd.DataFrame(every_food_numpy_vector,columns=vector_name_list)
#every_food_df_vector.to_csv("test.csv",index=0)
all_df_vector=pd.concat([ID_name_vector,every_food_df_vector],axis=1)
all_df_vector.to_csv("fasttext_allpeoplecut1230_cuttf1230_v17_w5_sg0_min3_i10_no_around_AVG.csv",index=0)

#直接轉TSV存檔
# b=pd.DataFrame(b)
# with open("route.tsv", 'w') as write_tsv:
#     write_tsv.write(b.to_csv(sep='\t', index=False,header=False))


f:\python3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [74]:
data=pd.read_csv("F:\資策會\專題\R跟jupyternotebook_code\complete.csv")
#①loc——通过行标签进行索引
#②iloc——通过行号进行索引
#③ix——结合前两种的混合索引
#data.loc[:,"Comp.1":"Comp.5"]     原本用loc[:,20:25]一直出不來,這邊列的位置不能空，跟R不一樣，至少要有個:
data_projector=data.iloc[:,19:24]
data_name=data.iloc[:,0:2]
data_fit=data.iloc[:,-1]
data_name_fit=pd.concat([data_name,data_fit],axis=1)

a=data_projector.columns
tmp=data_projector.as_matrix()
tmp_around=np.around(tmp,decimals=2)
data_projector_around=pd.DataFrame(tmp_around,columns=a)
with open("data_projector_around.tsv", 'w') as write_tsv:
    write_tsv.write(data_projector_around.to_csv(sep='\t', index=False,header=False))

with open("data_name_fit.tsv", 'w',encoding="utf-8") as write_tsv:
    write_tsv.write(data_name_fit.to_csv(sep='\t', index=False,header=True))

f:\python3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [73]:
data_fit=data.iloc[:,-1]
data_name_fit

,name,food_id,fit
0,陳皮紅豆湯圓,29102,1
1,起司醬佐松子核果義大利麵,29103,2
2,BBQ豬肋排,29104,3
3,韓式辣味油醋沙拉,29105,3
4,法式干邑烤布蕾,29106,3
...,...,...,...
6424,什錦海鮮凍,35526,1
6425,酸辣海鮮沙拉,35527,1
6426,醋溜鮭魚,35528,1
6427,德州式烤肩胛心,35529,3


In [4]:
data=pd.read_csv("F:\資策會\專題\R跟jupyternotebook_code\complete.csv")
#①loc——通过行标签进行索引
#②iloc——通过行号进行索引
#③ix——结合前两种的混合索引
#data.loc[:,"Comp.1":"Comp.5"]     原本用loc[:,20:25]一直出不來,這邊列的位置不能空，跟R不一樣，至少要有個:
data_projector=data.iloc[:,3:20]
data_name=data.iloc[:,0:2]
data_fit=data.iloc[:,-1]
data_name_fit=pd.concat([data_name,data_fit],axis=1)

a=data_projector.columns
tmp=data_projector.as_matrix()
tmp_around=np.around(tmp,decimals=2)
data_projector_around=pd.DataFrame(tmp_around,columns=a)
with open("data_projector_raw_around.tsv", 'w') as write_tsv:
    write_tsv.write(data_projector_around.to_csv(sep='\t', index=False,header=False))

# with open("data_name_fit.tsv", 'w',encoding="utf-8") as write_tsv:
#     write_tsv.write(data_name_fit.to_csv(sep='\t', index=False,header=True))

f:\python3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
